Reference

# Scrape SUUMO data

In [1]:
from retry import retry # automatic retry
import requests
from bs4 import BeautifulSoup # parse HTML
import pandas as pd
import gc
import time
import os
import json
import re

In [2]:
@retry(tries=3, delay=10, backoff=2)
def get_html(url):
    """
    Uses BeautifulSoup to parse HTML content from url. Retries 3 times with 10 second delay that doubles each try.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Will raise an exception for HTTP errors
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException:
        return None

## 1. Scrape URLs
- Scrape URLs for individual purchasable property from SUUMO.jp collection pages

In [3]:
if not os.path.exists("/app/data/used_property_urls.csv"):
    
    collection_pages = {
        "used_apartments": "https://suumo.jp/jj/bukken/ichiran/JJ012FC001/?ar=030&bs=011&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13106&sc=13107&sc=13108&sc=13109&sc=13110&sc=13111&sc=13112&sc=13113&sc=13114&sc=13115&sc=13116&sc=13117&sc=13118&sc=13119&sc=13120&sc=13121&sc=13122&sc=13123&ta=13&po=0&pj=1&pc=100",
        # "new_apartments": "https://suumo.jp/jj/bukken/ichiran/JJ011FC001/?ar=030&bs=010&ta=13&po=0&pj=1&initFlg=1&bknlistmodeflg=1&page=1&pc=100",
        "used_houses": "https://suumo.jp/jj/bukken/ichiran/JJ012FC001/?ar=030&bs=021&cn=9999999&cnb=0&ekTjCd=&ekTjNm=&hb=0&ht=9999999&kb=1&kt=9999999&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&tb=0&tj=0&tt=9999999&po=0&pj=1&pc=100",
        # "new_houses": "https://suumo.jp/jj/bukken/ichiran/JJ012FC001/?ar=030&bs=020&ekTjCd=&ekTjNm=&hb=0&ht=9999999&kb=1&km=1&kt=9999999&kw=1&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&tb=0&tj=0&tt=9999999&po=0&pj=1&pc=100"
    }

    list_urls = []
    page = 0

    for category, url in collection_pages.items():
        print(f"Category: {category}")

        while True:
            soup = get_html(url.format(page))
            
            if not soup:
                print(f"No soup in {url}")
                break 

            for h2 in soup.find_all('h2', class_='property_unit-title'):
                a_tag = h2.find('a', href=True) 
                if a_tag and a_tag['href']:
                    property_url = "https://suumo.jp" + a_tag['href']
                    list_urls.append((category, property_url))

            # Find the 'next' page link with text "次へ"
            next_page = soup.find('a', text="次へ")
            if next_page and next_page.get('href'):
                url = "https://suumo.jp" + next_page['href']
            else:
                print(f"No more {category} pages.")
                break

    df_urls = pd.DataFrame(list_urls, columns=['Category', 'URL'])
    df_urls.to_csv("/app/data/used_property_urls.csv", index=False)
    df_urls.head()


In [9]:
def parse_price(value):
    try:
        total_price = 0
        price_str = value.strip().split('円')[0]
        if "億" in price_str:
            oku_pattern = re.compile(r'(\d+)億')       # match int number before '億'
            oku_match = oku_pattern.search(price_str)
            if oku_match:
                total_price += int(oku_match.group(1)) * 100000000
        if "万" in price_str:
            man_pattern = re.compile(r'(\d+)万')
            man_match = man_pattern.search(price_str)
            if man_match:
                total_price += int(man_match.group(1)) * 10000
            return total_price
        else:
            return int(price_str)
    except ValueError:
        return None

def parse_area(value):
    try:
        area_pattern = re.compile(r'(\d+(\.\d+)?)m')   # match decimal number before 'm'
        area_match = area_pattern.search(value)
        if area_match:
            return float(area_match.group(1))
        else:
            return None
    except ValueError:
        return None

def extract_number(string):
    match = re.search(r'\d+', string)
    if match:
        return int(match.group())
    return None

def parse_year(value):
    try:
        return int(value.strip().split('年')[0])
    except ValueError:
        return None

def remove_whitespace(text):
    """ Remove white space characters suchs as newline, tab, etc. """
    return re.sub(r'\s+', ' ', text).strip()

def parse_features(soup):
    """ Get soup of data in the 特徴ピックアップ section. """
    features_section = soup.find('h3', string='特徴ピックアップ')
    if features_section:
        parent_div = features_section.find_parent('div', class_='secTitleOuterR')
        if parent_div == None:
            parent_div = features_section.find_parent('div', class_='secTitleOuterK')
        features_div = parent_div.find_next_sibling('div', class_='mt10')
        if features_div:
            features_text = ' '.join(features_div.stripped_strings)
            return remove_whitespace(features_text)
    return None

pd_urls = pd.read_csv("/app/data/used_property_urls.csv", index_col=False)
df_urls = pd.DataFrame(pd_urls, columns=['Category', 'URL'])
df_apt_urls = df_urls.loc[df_urls['Category'] == 'used_apartments']

list_data = []

count = 0
total = len(df_urls)
batch_size = 1000

list_data = []

if os.path.exists("/app/data/used_property_data.csv"):
    df_partial = pd.read_csv(f"/app/data/used_property_data.csv", index_col=False)
    # Extract URL sets
    urls_set1 = set(df_apt_urls['URL'])
    urls_set2 = set(df_partial['URL'])
    # URLs in df_apt_urls but not in df_partial
    unique_urls1 = list(urls_set1 - urls_set2)
else:
    unique_urls1 = list(df_apt_urls['URL'])

for url in unique_urls1:
    print(url)
    soup = get_html(url)

    if not soup:
        print(f"No soup in {url}")
        break 

    # Initialize dictionary to store the data
    property_data = {}
    property_data['url'] = url

    # Iterate over each row in the table
    target_table = soup.find('table', summary="表")
    for row in target_table.find_all('tr'):
        # Find all th and td pairs in the row
        ths = row.find_all('th')
        tds = row.find_all('td')
        
        for th, td in zip(ths, tds):
            header = th.text.strip()
            value = td.text.strip()

            if "価格" in header:
                property_data['price'] = parse_price(value)
            elif "間取り" in header:
                property_data['plan'] = value.strip()
            elif "専有面積" in header:
                property_data['area'] = parse_area(value)
            elif "その他面積" in header:
                property_data['balcony_area'] = parse_area(value)
            elif "所在階/構造・階建" in header:
                level_details = value.strip().split('/')
                property_data['level'] = extract_number(level_details[0])
                property_data['no_floors'] = extract_number(level_details[1])
            elif "完成時期（築年月）" in header:
                property_data['year'] = parse_year(value)
            elif "住所" in header:
                property_data['address'] = value.strip().split('\n')[0]

    property_data['features'] = parse_features(soup)

    if any(value is None for value in property_data.values()):
        print(f"{url} data has None.")
        print(property_data)
        break

    list_data.append(property_data)

    count += 1

    # Periodically save property_data and clear the list from memory
    if count % batch_size == 0:
        print(f"Processed {count}/{total} URLs")
        df_data = pd.DataFrame(list_data)
        df_merged = pd.merge(df_apt_urls, df_data, left_on='URL', right_on='url', how='inner')
        df_merged = df_merged.drop(columns=['url'])

        if not os.path.exists("/app/data/used_property_data.csv"):
            # First batch: Write header and data
            df_merged.to_csv(f'/app/data/used_property_data.csv', index=False)
        else:
            df_merged.to_csv(f'/app/data/used_property_data.csv', mode='a', header=False, index=False)

        property_data.clear()
        list_data = []
        gc.collect()  # Manually release memory
        time.sleep(5)  # Rest 5 seconds to reduce server load

        break

# df_merged = pd.read_csv(f"/app/data/used_property_data.csv", index_col=False)
# df_merged.head()



https://suumo.jp/ms/chuko/tokyo/sc_adachi/nc_75070928/
https://suumo.jp/ms/chuko/tokyo/sc_taito/nc_75117331/
Processed 2/25349 URLs


In [8]:
df_merged.head()

,Category,URL,price,plan,area,balcony_area,level,no_floors,year,address,features
0,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_adachi/nc_7...,11900000,2DK,41.34,2.43,2,4,1979,東京都足立区花畑５-2-9,年内引渡可 / 即引渡可 / 角住戸 / 陽当り良好 / 全居室収納 / シャワー付洗面化粧...
1,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_taito/nc_75...,13800000,1K,16.45,2.69,5,9,1991,東京都台東区松が谷３-６－２,瑕疵保証付（不動産会社独自） / ２沿線以上利用可 / 内装リフォーム / 駅まで平坦 / ...


ADD BALCONY AREA

In [ ]:
# # def get_property_data(url):
    

# pd_urls = pd.read_csv("/app/data/property_urls.csv", index_col=False)
# df_urls = pd.DataFrame(pd_urls, columns=['Category', 'URL'])
# df_urls.head()

# property_data = []

# count = 0
# total = len(df_urls)
# batch_size = 1000

# for url in df_urls['URL']:
#     soup = get_html(url)

#     if not soup:
#         print(f"No soup in {url}")
#         property_data.append({'URL': url, 
#                         '向き': "掲載終了", 
#                         '間取り詳細': "掲載終了", 
#                         '物件の構造': "掲載終了", 
#                         '物件の階建': "掲載終了",
#                         '築年月': "掲載終了", 
#                         '損保': "掲載終了", 
#                         '駐車場': "掲載終了", 
#                         '取引態様': "掲載終了",
#                         '条件': "掲載終了", 
#                         '総戸数': "掲載終了", 
#                         '契約期間': "掲載終了", 
#                         '仲介手数料': "掲載終了",
#                         '保証会社': "掲載終了", 
#                         '備考': "掲載終了", 
#                         '部屋の特徴': "掲載終了"})
#         break 

#     # Find the "Direction"
#     try:
#         element = soup.find("table", {"class": "property_view_table"})
#         direction = element.find('th', string="向き").find_next_sibling('td').get_text(strip=True)
#     except AttributeError:
#         direction = 'Null'
    
#     # Find the required data table
#     tbodys = soup.find("table", {"class": "data_table table_gaiyou"})
    
#     # Define a function to safely fetch the data for each field
#     def get_text_safe(th_string):
#         try:
#             return tbodys.find('th', string=th_string).find_next_sibling('td').get_text(strip=True)
#         except AttributeError:
#             return 'Null'
    
#     # Use the function to fetch data for each field
#     layout_detail = get_text_safe('間取り詳細')
#     structure = get_text_safe('構造')
#     floor = get_text_safe('階建')
#     yearmonth = get_text_safe('築年月')
#     insurance = get_text_safe('損保')
#     parking = get_text_safe('駐車場')
#     transaction_type = get_text_safe('取引態様')
#     conditions = get_text_safe('条件')
#     total_units = get_text_safe('総戸数')
#     lease_term = get_text_safe('契約期間')
#     intermediary_fee = get_text_safe('仲介手数料')
#     guarantee_company = get_text_safe('保証会社')
#     note = get_text_safe('備考')

#     # Find room features
#     try:
#         li_element = soup.find('div', {'id': 'bkdt-option'}).find('li')
#         feature = li_element.get_text(strip=True)
#     except AttributeError:
#         feature = 'Null'

#     # Append the data to the list
#     property_data.append({
#         'URL': url,
#         '向き': direction,
#         '間取り詳細': layout_detail,
#         '物件の構造': structure,
#         '物件の階建': floor,
#         '築年月': yearmonth,
#         '損保': insurance,
#         '駐車場': parking,
#         '取引態様': transaction_type,
#         '条件': conditions,
#         '総戸数': total_units,
#         '契約期間': lease_term,
#         '仲介手数料': intermediary_fee,
#         '保証会社': guarantee_company,
#         '備考': note,
#         '部屋の特徴': feature
#     })
    
#     # Update the counter and print the current progress
#     count += 1
#     print(f"Processed {count}/{total} URLs")

#     # Periodically save property_data and clear the list from memory
#     if count % batch_size == 0:
#         partial_df = pd.DataFrame(property_data)
#         partial_df.to_csv(f'property_data_part_{count // batch_size}.csv', index=False)
#         property_data.clear()
#         gc.collect()  # Manually release memory
#         time.sleep(5)  # Rest 5 seconds to reduce server load
#     partial_df = pd.DataFrame(property_data)
#     partial_df.head()
#     break


In [ ]:
partial_df.head()

,URL,向き,間取り詳細,物件の構造,物件の階建,築年月,損保,駐車場,取引態様,条件,総戸数,契約期間,仲介手数料,保証会社,備考,部屋の特徴
0,https://suumo.jp/ms/chuko/tokyo/sc_adachi/nc_7...,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null


In [ ]:
# for page in range(1, max_page + 1):
#     url = base_url.format(page) # Replace {} with page number
#     soup = get_html(url)        # Get parsed data

#     property_tags = soup.findAll("div", {"class": "cassetteitem"})  # All <div>'s with "cassetteitem" in the class attribute contain propetry info.
    
#     # print(f"Page: {page} - items: {property_tags}")
    
#     for property_tag in property_tags:
#         stations = property_tag.findAll("div", {"class": "cassetteitem_detail-text"})
        
#         # process each station
#         for station in stations:
#             # define variable 
#             base_data = {}

#             # collect base information    
#             # Name
#             base_data["名称"] = property_tag.find("div", {"class": "cassetteitem_content-title"}).getText().strip()
            
#             # category
#             base_data["カテゴリー"] = property_tag.find("div", {"class": "cassetteitem_content-label"}).getText().strip()

#             # address
#             base_data["アドレス"] = property_tag.find("li", {"class": "cassetteitem_detail-col1"}).getText().strip()

#             # nearby station
#             base_data["アクセス"] = station.getText().strip()

#             # building's age
#             base_data["築年数"] = property_tag.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()

#             # structure
#             base_data["構造"] = property_tag.find("li", {"class": "cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()
            
#             # process data for each room
#             tbodys = property_tag.find("table", {"class": "cassetteitem_other"}).findAll("tbody")
            
#             for tbody in tbodys:
#                 data = base_data.copy()

#                 # floor
#                 data["階数"] = tbody.findAll("td")[2].getText().strip()

#                 # rent
#                 data["家賃"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()

#                 # Management fee
#                 data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()

#                 # deposit
#                 data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()

#                 #gratuity fee
#                 data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()

#                 #layout
#                 data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()

#                 #area(m^2)
#                 data["面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()

#                 data["URL"] = "https://suumo.jp" + tbody.findAll("td")[8].find("a").get("href")
                
#                 all_data.append(data)
# df = pd.DataFrame(all_data)
# df.head()